### If you dont want to use Wandb, disable Wandb otherwise optional

references for WANDB
https://analyticsindiamag.com/hands-on-guide-to-weights-and-biases-wandb-with-python-implementation/

https://docs.wandb.ai/


In [36]:
!nvidia-smi

Sun Nov 23 12:07:54 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:17:00.0 Off |                    0 |
| N/A   54C    P0              70W / 300W |  12681MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [37]:
import os
print(os.getcwd())
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

/workspace/Approach2


In [38]:
!huggingface-cli login --token {hf_token}

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `llm_finetuning` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `llm_finetuning`


In [39]:
import os
os.environ["WANDB_DISABLED"]="true"

# Install required packages

In [40]:
# !pip install transformers==4.50.3

In [41]:
import transformers
print(transformers.__version__)

4.50.3


### Loading the test dataset

In [7]:
# !pip install datasets evaluate

In [13]:
from datasets import load_dataset
from evaluate import load
raw_datasets = load_dataset("thenlpresearcher/test_data_marathi")
metric = load("sacrebleu")

In [43]:
import os
print(os.getcwd())

/workspace/Approach2


The dataset object itself is [datasetdict](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [44]:
raw_datasets

DatasetDict({
    test: Dataset({
        features: ['Unnamed: 0', 'punct_type', 'sent_written', 'sent_meant', 'gt_marathi', 'gemini_out', 'cfilt_out'],
        num_rows: 54
    })
})

In [45]:
raw_datasets["test"][0]

{'Unnamed: 0': 0,
 'punct_type': 'Comma',
 'sent_written': 'Chanting the choir raised the volume as the celebrant intoned the prayer.',
 'sent_meant': 'Chanting, the choir raised the volume as the celebrant intoned the prayer.',
 'gt_marathi': 'जल्लोष करणाऱ्या व्यक्तीने प्रार्थनेचा उच्चार करताच, गायनाने आवाज वाढवला.',
 'gemini_out': 'धर्मगुरू प्रार्थना म्हणत असताना, घोष करणाऱ्या गायकवृंदाने आवाज वाढवला.',
 'cfilt_out': 'उत्सवी प्रार्थनेचा उच्चार करत असताना, जप करत, गायकवृंदाने आवाज वाढवला.'}

To get a sense of how the data looks like, the following function will show some examples picked randomly in the dataset.

In [11]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
show_random_elements(raw_datasets["test"])

,Unnamed: 0,punct_type,sent_written,sent_meant,gt_marathi,gemini_out,cfilt_out
0,6,Comma,Executors delay giving information about substantial deviations from agreed dates. Because of this action cannot be taken in time.,"Executors delay giving information about substantial deviations from agreed dates. Because of this, action cannot be taken in time.","मान्य केलेल्या तारखांपासून लक्षणीय विचलनांची माहिती देण्यास कार्यकारी विलंब करतात. यामुळे, वेळेवर कारवाई केली जाऊ शकत नाही.",अंमलबजावणी करणारे मान्य केलेल्या तारखांपासून महत्त्वपूर्ण विचलनाबद्दल माहिती देण्यास विलंब करतात. यामुळे वेळेत कारवाई करता येत नाही.,मान्य केलेल्या तारखांपासून लक्षणीय विचलनांची माहिती देण्यास कार्यकारी विलंब करतात. यामुळे वेळेवर कारवाई होऊ शकत नाही.
1,50,Parenthesis,"The glass content of this material is low, 10% by weight, for this application.","The glass content of this material is low, (10% by weight), for this application.",या अनुप्रयोगासाठी या सामग्रीचे काचेचे प्रमाण कमी (वजनानुसार 10 टक्के) आहे.,या सामग्रीतील काचेचे प्रमाण (वजनानुसार १०%) या उपयोगासाठी कमी आहे.,या अनुप्रयोगासाठी या सामग्रीचे काचेचे प्रमाण कमी आहे (वजनानुसार 10 टक्के).
2,49,Quotes,One should read G. V. Carey’s chapter Proof Correction in Mind the Stop.,One should read G. V. Carey’s chapter ‘Proof Correction’ in Mind the Stop.,जी. व्ही. कॅरी यांचा'प्रूफ करेक्शन'हा अध्याय'माईंड द स्टॉप'मध्ये वाचला पाहिजे.,जी. व्ही. कॅरीचा 'माईंड द स्टॉप' मधील 'प्रूफ करेक्शन' हा अध्याय वाचला पाहिजे.,माइंड द स्टॉप मधील जी. व्ही. कॅरी यांचा 'प्रूफ करेक्शन' हा अध्याय वाचला पाहिजे.
3,44,Comma,Connection to PTT-supplied packet-switch networks will be a prime requirement of the workstation and gateways into these networks are planned by the PTTs to ensure seamless data communication.,"Connection to PTT-supplied packet-switch networks will be a prime requirement of the workstation, and gateways into these networks are planned by the PTTs to ensure seamless data communication.",पी. टी. टी.-पुरविलेल्या पॅकेट-स्विच नेटवर्कशी जोडणी ही वर्कस्टेशनची प्रमुख आवश्यकता असेल आणि अखंड डेटा संप्रेषण सुनिश्चित करण्यासाठी पी. टी. टी. द्वारे या नेटवर्कमधील प्रवेशद्वारांचे नियोजन केले जाते.,पीटीटीने पुरवलेल्या पॅकेट-स्विच नेटवर्कशी जोडणी ही वर्कस्टेशनची प्रमुख आवश्यकता असेल आणि अखंड डेटा संवाद सुनिश्चित करण्यासाठी या नेटवर्कमध्ये प्रवेशासाठी गेटवे पीटीटीद्वारे नियोजित आहेत.,पीटीटी-पुरविलेल्या पॅकेट-स्विच नेटवर्कशी जोडणी ही वर्कस्टेशनची प्रमुख आवश्यकता असेल आणि या नेटवर्कमधील प्रवेशद्वारांची योजना पीटीटीद्वारे अखंड डेटा संप्रेषण सुनिश्चित करण्यासाठी आखली जाते.
4,36,Comma,Insert the new disk into the disk drive with the notch at the bottom facing the drive’s entry slot.,"Insert the new disk into the disk drive, with the notch at the bottom facing the drive’s entry slot.","डिस्क ड्राइव्हमध्ये नवीन डिस्क घाला, ज्यात ड्राइव्हच्या प्रवेश स्लॉटच्या समोर तळाशी खाच आहे.","नवीन डिस्क डिस्क ड्राइव्हमध्ये घाला, ज्याचा तळाशी असलेला नॉच ड्राइव्हच्या प्रवेश स्लॉटकडे तोंड करून असेल.","डिस्क ड्राइव्हमध्ये नवीन डिस्क समाविष्ट करा, ज्याच्या तळाशी नोच ड्राइव्हच्या एंट्री स्लॉटच्या समोर आहे."


In [7]:
def initialize_model(ckpt_dir, device, quantization=None):
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig
    import torch

    # Quantization setup
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

#     # Load tokenizer
#     tokenizer = AutoTokenizer.from_pretrained(ckpt_dir, trust_remote_code=True)

    # Load model
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        quantization_config=qconfig,
    )

    # Move to device and optionally convert to half precision
    if qconfig is None:
        model = model.to(device)
        if device == "cuda":
            model.half()

    # Make sure model is in training mode for fine-tuning
    model.eval()

    return model

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig

en_indic_ckpt_dir = "thenlpresearcher/tmp-test-indictrans"
model_checkpoint = "ai4bharat/indictrans2-en-indic-dist-200M"

device = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None
en_indic_tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [53]:
!pip install "transformers>=4.50.3"

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [8]:
# Load model directly
from transformers import AutoModelForSeq2SeqLM
model = initialize_model(en_indic_ckpt_dir, device)

Then we just need to pass all of this along with our datasets to the Seq2SeqTrainer:

In [9]:
tokenizer = en_indic_tokenizer

In [21]:
raw_datasets['test']

Dataset({
    features: ['Unnamed: 0', 'punct_type', 'sent_written', 'sent_meant', 'gt_marathi', 'gemini_out', 'cfilt_out'],
    num_rows: 54
})

In [10]:
from IndicTransToolkit.processor import IndicProcessor

In [11]:
ip = IndicProcessor(inference=True)

In [14]:
BATCH_SIZE = 4
src_lang, tgt_lang = "eng_Latn", "mar_Deva"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# -------------------- LOAD DATA --------------------
src_sentences = raw_datasets['test']["sent_written"]
ref_gt     = raw_datasets['test']["gt_marathi"]
ref_gem    = raw_datasets['test']["gemini_out"]
ref_cfilt  = raw_datasets['test']["cfilt_out"]

In [15]:
print(len(src_sentences))
print(len(ref_gt))
print(len(ref_gem))
print(len(ref_cfilt))

54
54
54
54


In [32]:
print(ip.preprocess_batch(["Chanting the choir raised…", "kaustubh is singing", "Snickers is tasty"], src_lang="eng_Latn", tgt_lang="mar_Deva"))

['eng_Latn mar_Deva Chanting the choir raised ...', 'eng_Latn mar_Deva kaustubh is singing', 'eng_Latn mar_Deva Snickers is tasty']


In [16]:
import torch

def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip, device, batch_size=8, max_length=256):
    """
    Translate a batch of sentences using a seq2seq model like IndicTrans with safety checks.

    Args:
        input_sentences (list of str): Source sentences to translate.
        src_lang (str): Source language code, e.g., "eng_Latn".
        tgt_lang (str): Target language code, e.g., "mar_Deva".
        model: Hugging Face seq2seq model.
        tokenizer: Corresponding tokenizer.
        ip: Preprocessing object (IndicProcessor).
        device: torch device ("cuda" or "cpu").
        batch_size (int): Batch size for generation.
        max_length (int): Maximum length of generated sequence.

    Returns:
        translations (list of str): Translated sentences.
    """
    model.to(device)
    model.eval()
    translations = []

    # Safe access for decoder_start_token_id
    decoder_start_token_id = getattr(model.config, "decoder_start_token_id", None)
    pad_token_id = getattr(tokenizer, "pad_token_id", None)
    eos_token_id = getattr(tokenizer, "eos_token_id", None)

    if decoder_start_token_id is None:
        print("[Warning] decoder_start_token_id is None. Using default generation behavior.")

    for i in range(0, len(input_sentences), batch_size):
        batch = input_sentences[i : i + batch_size]
        print('here')

        # Preprocess the batch
        batch_preprocessed = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)
        print(batch_preprocessed)
        if not isinstance(batch_preprocessed, list) or len(batch_preprocessed) == 0:
            print(f"[Warning] Preprocessed batch is empty at index {i}. Skipping...")
            continue

#         Debug: print first 2 sentences after preprocessing
        print(f"[Debug] Preprocessed batch sample: {batch_preprocessed[:2]}")

        # Tokenize
        inputs = tokenizer(
            batch_preprocessed,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        )

        # Move tensors to the correct device
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Generate translations with safety parameters
        with torch.no_grad():
            try:
                generated_tokens = model.generate(
                    **inputs,
                    use_cache=True,
                    min_length=5,
                    max_length=max_length,
                    num_beams=5,
                    num_return_sequences=1,
                    early_stopping=True,
                    decoder_start_token_id=decoder_start_token_id,
                    pad_token_id=pad_token_id,
                    eos_token_id=eos_token_id
                )
            except Exception as e:
                print(f"[Error] Generation failed for batch starting at index {i}: {e}")
                continue

        # Decode generated tokens
        decoded_texts = tokenizer.batch_decode(
            generated_tokens,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )

#         Debug: print first 2 decoded outputs
        print(f"[Debug] Decoded sample: {decoded_texts[:2]}")

        # Postprocess translations
        try:
            postprocessed = ip.postprocess_batch(decoded_texts, lang=tgt_lang)
            translations += postprocessed
        except Exception as e:
            print(f"[Error] Postprocessing failed for batch starting at index {i}: {e}")
            translations += decoded_texts  # fallback

        # Free GPU memory
        del inputs, generated_tokens
        torch.cuda.empty_cache()

    return translations

In [17]:
src_lang, tgt_lang = "eng_Latn", "mar_Deva"

prefix = f"{tgt_lang} {src_lang}"

def remove_prefix(text):
    if text.startswith(prefix):
        return text[len(prefix):].strip()
    return text.strip()

In [21]:
import os
print(os.getcwd())

/workspace/Approach2


In [24]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_path = "tmp-test-indictrans"

model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

The repository for tmp-test-indictrans contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/tmp-test-indictrans.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


In [25]:
from evaluate import load
from tqdm import tqdm
import pandas as pd

# -------------------- TRANSLATION --------------------
valid_src = []
valid_pred = []
valid_gt = []
valid_gem = []
valid_cfilt = []

# Translate in batches
all_translations = []
for i in tqdm(range(0, len(src_sentences), BATCH_SIZE)):
    batch = src_sentences[i:i+BATCH_SIZE]
#     print(batch)
    translations = batch_translate(
        batch,
        src_lang,
        tgt_lang,
        model,
        tokenizer,
        ip,
        device=DEVICE
    )
    all_translations.extend(translations)

    if translations is None:
        print(f"[SKIPPED] Batch {i}: Returned None")
        continue

    cleaned = [remove_prefix(t) for t in translations]

    valid_src.extend(batch)
    valid_pred.extend(cleaned)
    valid_gt.extend(ref_gt[i:i + len(batch)])
    valid_gem.extend(ref_gem[i:i + len(batch)])
    valid_cfilt.extend(ref_cfilt[i:i + len(batch)])

print(f"\nSuccessful translations: {len(valid_pred)} / {len(src_sentences)}")

  0%|                                                | 0/14 [00:00<?, ?it/s]

here
['eng_Latn mar_Deva Chanting the choir raised the volume as the celebrant intoned the prayer .', 'eng_Latn mar_Deva A six-month-old calf was submitted for examination , showing lameness in all four legs which had been present since soon after birth .', 'eng_Latn mar_Deva Planning authorities should provide alternative locations for small businesses which are or would be offensive in a residential area .', 'eng_Latn mar_Deva As the machine develops the forms we use to record data from past projects will be amended .']
[Debug] Preprocessed batch sample: ['eng_Latn mar_Deva Chanting the choir raised the volume as the celebrant intoned the prayer .', 'eng_Latn mar_Deva A six-month-old calf was submitted for examination , showing lameness in all four legs which had been present since soon after birth .']


  7%|██▊                                     | 1/14 [00:03<00:48,  3.74s/it]

[Debug] Decoded sample: ['" " " " । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । ।', '" " " " । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । ।

 14%|█████▋                                  | 2/14 [00:07<00:42,  3.52s/it]

[Debug] Decoded sample: ['" " " " । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । ।', '" " " " । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । । ।

 14%|█████▋                                  | 2/14 [00:09<00:57,  4.80s/it]


KeyboardInterrupt: 

In [26]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor
# recommended to run this on a gpu with flash_attn installed
# don't set attn_implemetation if you don't have flash_attn
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

src_lang, tgt_lang = "eng_Latn", "hin_Deva"
model_name = "ai4bharat/indictrans2-en-indic-dist-200M"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

ip = IndicProcessor(inference=True)

input_sentences = [
    "When I was young, I used to go to the park every day.",
    "We watched a new movie last week, which was very inspiring.",
    "If you had met me at that time, we would have gone out to eat.",
    "My friend has invited me to his birthday party, and I will give him a gift.",
]

batch = ip.preprocess_batch(input_sentences, src_lang=src_lang, tgt_lang=tgt_lang)

# Tokenize the sentences and generate input encodings
inputs = tokenizer(
    batch,
    truncation=True,
    padding="longest",
    return_tensors="pt",
    return_attention_mask=True,
).to(DEVICE)

# Generate translations using the model
with torch.no_grad():
    generated_tokens = model.generate(
        **inputs,
        use_cache=True,
        min_length=0,
        max_length=256,
        num_beams=5,
        num_return_sequences=1,
    )

# Decode the generated tokens into text
generated_tokens = tokenizer.batch_decode(
    generated_tokens,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True,
)

# Postprocess the translations, including entity replacement
translations = ip.postprocess_batch(generated_tokens, lang=tgt_lang)

for input_sentence, translation in zip(input_sentences, translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")


eng_Latn: When I was young, I used to go to the park every day.
hin_Deva: जब मैं छोटा था, मैं हर दिन पार्क जाता था।
eng_Latn: We watched a new movie last week, which was very inspiring.
hin_Deva: हमने पिछले हफ्ते एक नई फिल्म देखी, जो बहुत प्रेरणादायक थी।
eng_Latn: If you had met me at that time, we would have gone out to eat.
hin_Deva: अगर आप उस समय मुझसे मिलते तो हम बाहर खाना खाने जाते।
eng_Latn: My friend has invited me to his birthday party, and I will give him a gift.
hin_Deva: मेरे दोस्त ने मुझे अपने जन्मदिन की पार्टी में आमंत्रित किया है, और मैं उसे एक उपहार दूंगा।


In [3]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor
# recommended to run this on a gpu with flash_attn installed
# don't set attn_implemetation if you don't have flash_attn
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

src_lang, tgt_lang = "eng_Latn", "mar_Deva"
model_name = "thenlpresearcher/iitb-en-indic-only-punct"
tokenizer_name =  "ai4bharat/indictrans2-en-indic-dist-200M"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trust_remote_code=True)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

ip = IndicProcessor(inference=True)

input_sentences = [
    "When I was young, I used to go to the park every day.",
    "We watched a new movie last week, which was very inspiring.",
    "If you had met me at that time, we would have gone out to eat.",
    "My friend has invited me to his birthday party, and I will give him a gift.",
]

batch = ip.preprocess_batch(input_sentences, src_lang=src_lang, tgt_lang=tgt_lang)

# Tokenize the sentences and generate input encodings
inputs = tokenizer(
    batch,
    truncation=True,
    padding="longest",
    return_tensors="pt",
    return_attention_mask=True,
).to(DEVICE)

# Generate translations using the model
with torch.no_grad():
    generated_tokens = model.generate(
        **inputs,
        use_cache=True,
        min_length=0,
        max_length=256,
        num_beams=5,
        num_return_sequences=1,
    )

# Decode the generated tokens into text
generated_tokens = tokenizer.batch_decode(
    generated_tokens,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True,
)

# Postprocess the translations, including entity replacement
translations = ip.postprocess_batch(generated_tokens, lang=tgt_lang)

for input_sentence, translation in zip(input_sentences, translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")


A new version of the following files was downloaded from https://huggingface.co/thenlpresearcher/iitb-en-indic-only-punct:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/thenlpresearcher/iitb-en-indic-only-punct:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


eng_Latn: When I was young, I used to go to the park every day.
mar_Deva: mar_Deva eng_Latn मी लहान होतो तेव्हा मी रोज उद्यानात जायचो.
eng_Latn: We watched a new movie last week, which was very inspiring.
mar_Deva: mar_Deva eng_Latn आम्ही गेल्या आठवड्यात एक नवीन चित्रपट पाहिला, जो खूप प्रेरणादायी होता.
eng_Latn: If you had met me at that time, we would have gone out to eat.
mar_Deva: mar_Deva eng_Latn त्यावेळी तुम्ही मला भेटलात असते तर आम्ही जेवायला बाहेर गेलो असतो.
eng_Latn: My friend has invited me to his birthday party, and I will give him a gift.
mar_Deva: mar_Deva eng_Latn माझ्या मित्राने मला त्याच्या वाढदिवसाच्या पार्टीला बोलावले आहे आणि मी त्याला भेट देईन.


In [ ]:
valid_pred[2]

In [29]:
mode = "only_punct"

In [30]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": valid_src,
    "prediction": valid_pred,
    "gt": valid_gt,
    "gemini": valid_gem,
    "cfilt": valid_cfilt
})

results_df.to_csv(f"{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {mode}_outputs.csv")

# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1, ref2, ref3):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1, r2, r3] for r1, r2, r3 in zip(ref1, ref2, ref3)]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score, chrf_score = compute_scores(valid_pred, valid_gt, valid_gem, valid_cfilt)

# Determine best reference per metric (based on BLEU)
all_scores = {
    "GT":    bleu.compute(predictions=valid_pred, references=[[r] for r in valid_gt])["score"],
    "Gemini": bleu.compute(predictions=valid_pred, references=[[r] for r in valid_gem])["score"],
    "CFILT":  bleu.compute(predictions=valid_pred, references=[[r] for r in valid_cfilt])["score"]
}

best_ref = max(all_scores, key=all_scores.get)

print("\n===== FINAL METRICS =====")
print(f"All references combined → BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}")
print(f"GT Marathi → BLEU: {all_scores['GT']:.2f}")
print(f"Gemini    → BLEU: {all_scores['Gemini']:.2f}")
print(f"CFILT     → BLEU: {all_scores['CFILT']:.2f}")
print(f"\n🎯 BEST REFERENCE = {best_ref} (by highest BLEU)")

# -------------------- SAVE METRICS --------------------
with open(f"punct_{mode}_indictrans2_eval_metrics.txt", "w") as f:
    f.write(f"All references combined → BLEU {bleu_score:.2f}, chrF++ {chrf_score:.2f}\n")
    f.write(f"GT    BLEU {all_scores['GT']:.2f}\n")
    f.write(f"Gem   BLEU {all_scores['Gemini']:.2f}\n")
    f.write(f"CFILT BLEU {all_scores['CFILT']:.2f}\n")
    f.write(f"\nBEST REFERENCE = {best_ref}\n")

print(f"Metrics written to punct_{mode}_baseline_outputs_eval_metrics.txt")

✔ Saved predictions to combined_outputs.csv

===== FINAL METRICS =====
All references combined → BLEU: 0.03, chrF++: 0.01
GT Marathi → BLEU: 0.02
Gemini    → BLEU: 0.02
CFILT     → BLEU: 0.02

🎯 BEST REFERENCE = Gemini (by highest BLEU)
Metrics written to punct_combined_baseline_outputs_eval_metrics.txt


In [33]:
results_df['prediction'][0]

'उत्सवी गायकांनी प्रार्थनेचा उच्चार केल्याने गायकवृंदाचा जप केल्याने आवाज वाढला.'